In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [422]:
data = pd.read_excel('../game_scraping/scraping_results.xlsx', index_col=0)

In [423]:
data['Week'].unique()

data['Series'] = 1

# Initialize the series counter
series_counter = 1

# Iterate through the dataframe to update the Series column
for i in range(1, len(data)):
    if data.loc[i, 'Week'] == 1 and data.loc[i-1, 'Week'] != 1:
        series_counter += 1
    data.loc[i, 'Series'] = series_counter

data['revenue_unit'] = data['Price'] * data['Sales']

data['previous_sales'] = data.groupby('Series')['Sales'].shift(1)
data['previous_price'] = data.groupby('Series')['Price'].shift(1)
data['change_price'] = data.groupby('Series')['Price'].diff()
data['remaining_previous'] = data.groupby('Series')['Remaining'].shift(1)
data

,Week,Price,Sales,Remaining,foresight,Choice,diff,revenue,perfect,Series,revenue_unit,previous_sales,previous_price,change_price,remaining_previous
0,1,60,87,1913,"Your revenue: $85,752, Perfect foresight strat...",0,15.8,85752,101892,1,5220,NaN,NaN,NaN,NaN
1,2,60,74,1839,"Your revenue: $85,752, Perfect foresight strat...",0,15.8,85752,101892,1,4440,87.0,60.0,0.0,1913.0
2,3,54,147,1692,"Your revenue: $85,752, Perfect foresight strat...",1,15.8,85752,101892,1,7938,74.0,60.0,-6.0,1839.0
3,4,54,45,1647,"Your revenue: $85,752, Perfect foresight strat...",0,15.8,85752,101892,1,2430,147.0,54.0,0.0,1692.0
4,5,48,160,1487,"Your revenue: $85,752, Perfect foresight strat...",2,15.8,85752,101892,1,7680,45.0,54.0,-6.0,1647.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1495,11,36,100,0,"Your revenue: $82,860, Perfect foresight strat...",0,20.0,82860,103596,100,3600,206.0,36.0,0.0,100.0
1496,12,36,0,0,"Your revenue: $82,860, Perfect foresight strat...",0,20.0,82860,103596,100,0,100.0,36.0,0.0,0.0
1497,13,36,0,0,"Your revenue: $82,860, Perfect foresight strat...",0,20.0,82860,103596,100,0,0.0,36.0,0.0,0.0
1498,14,36,0,0,"Your revenue: $82,860, Perfect foresight strat...",0,20.0,82860,103596,100,0,0.0,36.0,0.0,0.0


In [417]:
from sklearn.linear_model import LinearRegression   
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, r2_score, mean_absolute_percentage_error

linear_model = LinearRegression()
linear_model.fit(data[['Week','Price']], data[['Sales']])
preds = linear_model.predict(data[['Week','Price']])

print('R2 score:',r2_score(data[['Sales']], preds))
print('MAE:',mean_absolute_error(data[['Sales']], preds))
print('MAPE:',mean_absolute_percentage_error(data[['Sales']], preds))   
print(linear_model.coef_)

R2 score: 0.3171610790613203
MAE: 62.95957184179123
MAPE: 6.655982285142552e+16
[[-26.81353903 -12.52288096]]


In [424]:
### More variables and boosting model


from xgboost import XGBRegressor

#### Train and test split

train = data[data['Series'] <= 80]
test = data[data['Series'] > 80]

model = XGBRegressor(n_estimators=500, learning_rate=0.01, random_state=123, max_depth = 3)
model.fit(train[['Week','Price','previous_sales','previous_price','change_price','remaining_previous']], train['Sales'])
preds = model.predict(test[['Week','Price','previous_sales','previous_price','change_price','remaining_previous']])

print('R2 score:',r2_score(test[['Sales']], preds))
print('MAE:',mean_absolute_error(test[['Sales']], preds))
print('MAPE:',mean_absolute_percentage_error(test[['Sales']], preds))


R2 score: 0.7010067543411818
MAE: 34.11835731665293
MAPE: 2929533032948476.5


In [82]:
test.head()

,Week,Price,Sales,Remaining,foresight,Choice,diff,revenue,perfect,Series,revenue_unit,previous_sales,previous_price,change_price,remaining_previous
1200,1,60,87,1913,"Your revenue: $84,354, Perfect foresight strat...",0,17.7,84354,102558,81,5220,NaN,NaN,NaN,NaN
1201,2,60,64,1849,"Your revenue: $84,354, Perfect foresight strat...",0,17.7,84354,102558,81,3840,87.0,60.0,0.0,1913.0
1202,3,54,141,1708,"Your revenue: $84,354, Perfect foresight strat...",1,17.7,84354,102558,81,7614,64.0,60.0,-6.0,1849.0
1203,4,54,132,1576,"Your revenue: $84,354, Perfect foresight strat...",0,17.7,84354,102558,81,7128,141.0,54.0,0.0,1708.0
1204,5,48,200,1376,"Your revenue: $84,354, Perfect foresight strat...",2,17.7,84354,102558,81,9600,132.0,54.0,-6.0,1576.0


In [456]:
from gurobipy import Model, GRB
import gurobipy as gp


pricing_model = Model('pricing')

### Decision variables 

prices_decision = pricing_model.addVars(data['Week'].unique(), lb=0, ub = 3, vtype=GRB.INTEGER, name='prices')

### Objective function
def obj(decisions):
    ## Decisions is a list of len = 15
    initial_price = 60
    allowed_prices = [60, 54, 48, 36]
    initial_stock = 2000

    prices = []
    stock_evolution = []
    revenue_evolution = []
    sales_evolution = []    
    for week in range(1,16):
        print(decisions)
        decision = decisions[week].getAttr('x')
        if week == 1: 
            tobe_predicted = np.array([[week, initial_price, np.nan, np.nan, np.nan, np.nan]])
            sales_predicted = np.floor(model.predict(tobe_predicted))
            new_remaining = initial_stock - sales_predicted[0]
            stock_evolution.append(new_remaining)
            old_price = initial_price
            prices.append(initial_price)
            revenue = sales_predicted[0] * initial_price
            revenue_evolution.append(revenue)
            sales_evolution.append(sales_predicted[0])
            
        else:
            new_price = allowed_prices[decision]
            
            if new_price <= old_price:
                new_price = new_price
            else:
                new_price = old_price
            tobe_predicted = np.array([[week, new_price, sales_predicted[0], old_price, new_price - old_price, new_remaining]])
            sales_predicted = np.floor(model.predict(tobe_predicted))
            if sales_predicted[0] > new_remaining:
                sales_predicted[0] = new_remaining
            new_remaining = new_remaining - sales_predicted[0]
            revenue = sales_predicted[0] * new_price
            sales_evolution.append(sales_predicted[0])
            revenue_evolution.append(revenue)
            stock_evolution.append(new_remaining)
            old_price = new_price
            prices.append(new_price)

    total_revenue = sum(revenue_evolution)
    return total_revenue

pricing_model.setObjective(obj(prices_decision), GRB.MAXIMIZE)

pricing_model.optimize()

AttributeError: Index out of range for attribute 'X'